In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
from groq import Groq
from pinecone import Pinecone, ServerlessSpec

In [12]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", 
    metric="cosine",
    spec = ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    ),
    #dimension=768,
    dimension=1536
    )

In [21]:
import json
with open("reviews.json") as file:
    data = json.load(file)
data['reviews']

[{'professor': 'Dr. Emily Hartwell',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Engaging lectures and challenging assignments. Dr. Hartwell really knows her stuff!'},
 {'professor': 'Prof. Michael Chang',
  'subject': 'Literature',
  'stars': 5,
  'review': "Absolutely brilliant! Prof. Chang's passion for literature is contagious."},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Biology',
  'stars': 3,
  'review': 'Decent instructor, but the course material could be more organized.'},
 {'professor': 'Prof. Robert Garcia',
  'subject': 'History',
  'stars': 4,
  'review': 'Fascinating insights into historical events. Prof. Garcia makes history come alive.'},
 {'professor': 'Dr. Lisa Thompson',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Difficult to follow lectures. More office hours would be helpful.'},
 {'professor': 'Prof. David Wilson',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Exceptional teacher. Prof. Wilson breaks down complex concepts with ease

In [36]:
processed_data = []
client = Groq()
for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review['professor'],
        "metadata": {
            "review": review['review'],
            "subject": review['subject'],
            "stars": review['stars']
        }
    })

NotFoundError: Error code: 404 - {'error': {'message': 'The model `text-embedding-3-small` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'code': 'model_not_found'}}